# Solving the Max-Cut Problem using Quantum Annealing

In [1]:
from collections import defaultdict

from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite
import networkx as nx
import dwave.inspector

import matplotlib
matplotlib.use("agg")
from matplotlib import pyplot as plt

## The Max-Cut problem

Given an undirected graph $G(V, E)$, where $V$ is the set of vertices and $E$ the set of edges, we want to find a partition of $V$ into two subsets $S_1 \subset V$ and $S_2 = V \setminus S_1$ such that the number of edges between $S_1$ and $S_2$ (i.e. the size of the cut-set) is a large as possible. Note that each edge in the cut-set has one endpoint in $S_1$ and one in $S_2$. For weighted undirected graphs, we want to maximize the sum of the weights on the edges in the cut-set.

### QUBO formulation of the Max-Cut problem

To solve the Max-Cut problem using quantum annealing, we'll model it in terms of a QUBO (quadratic unconstrained binary optimization) problem

$$
    \mathrm{Minimize}\quad f(\mathbf{x}) = \mathbf{x}^\top \mathbf{Q} \mathbf{x},
$$

where $\mathbf{x} = \{0, 1\}^n$ is a vector of binary variables and $\mathbf{Q}$ is a user-specified $n \times n$ coefficient matrix.

Let us assign a binary variable $x_i \in \{0, 1\},\ i = 1, ..., |V|$ to each of the vertices $i \in V$ of our undirected graph $G$. We'll use those binary variables to indicate whether $i$ is in the subset $S_1$ or $S_2$:

$$
    x_i = \begin{cases}
        1, & i \in S_1 \\
        0, & i \in S_2
    \end{cases}
$$

Due to the binary nature of the model, the value of the expression $x_i + x_j - 2x_ix_j$ can be used to indicate whether the edge $\{i, j\}$ belongs to the cut-set:

$$
    x_i + x_j - 2x_ix_j = \begin{cases}
        1, & \textrm{edge } \{i, j\}\, \textrm{is in the cut-set}, \\
        0, & \textrm{edge } \{i, j\}\, \textrm{is not in the cut-set}
    \end{cases}
$$

Finding the max-cut between $S_1$ i $S_2$ is therefore equivalent to maximizing the sum of the above expressions for each edge of the edge-set $E$:

$$
    \operatorname{Maximize}\quad f(\mathbf{x}) = \sum_{(i, j) \in E} (x_i + x_j - 2x_ix_j),
$$

or, when converted to a minimization problem:
$$
    \operatorname{Minimize}\quad f(\mathbf{x}) = \sum_{(i, j) \in E} (2x_ix_j - x_i - x_j),
$$
which is a QUBO problem for $\mathbf{x} = \{0, 1\}^{|V|}$.

A similar procedure is used for weighted undirected graph. Since all edges have weights assigned to them, one needs to multilpy the expression $x_i + x_j - 2x_ix_j$ with the corresponding weight $w_{ij}$:
$$
    w_{ij} (x_i + x_j - 2x_ix_j) = \begin{cases}
        w_{ij}, & \textrm{edge } \{i, j\}\, \textrm{is in the cut-set}, \\
        0, & \textrm{edge } \{i, j\}\, \textrm{is not in the cut-set},
    \end{cases}
$$
the QUBO problem is then:
\begin{equation}
    \operatorname{Minimize}\quad f(\mathbf{x}) = \sum_{(i, j) \in E} w_{ij} (2x_ix_j - x_i - x_j)
\end{equation}

## QA Max-Cut: Example

The example is taken from D-Wave's [maximum-cut](https://github.com/dwave-examples/maximum-cut) GitHub repository.

In [58]:
G = nx.Graph()
G.add_edges_from([(1,2),(1,3),(2,4),(3,4),(3,5),(4,5)])

In [59]:
nx.draw(G, with_labels=True)
plt.savefig('simple_graph.png')

![Simple graph](simple_graph.png)

Construct the $\mathbf{Q}$ matrix:

In [60]:
Q = defaultdict(int)

for i, j in G.edges:
    Q[i, i] -= 1
    Q[j, j] -= 1
    Q[i, j] += 2

Set up QPU parameters:

In [61]:
chainstrength = 8
numruns = 10

Run the QUBO:

In [62]:
sampler = EmbeddingComposite(DWaveSampler())
response = sampler.sample_qubo(Q,
                               chain_strength=chainstrength,
                               num_reads=numruns,
                               label='Example - Maximum Cut')

In [63]:
dwave.inspector.show(response);

In [64]:
print(response)

   1  2  3  4  5 energy num_oc. chain_.
0  1  0  0  1  1   -5.0       3     0.0
1  0  1  1  0  1   -5.0       2     0.0
2  0  1  1  0  0   -5.0       4     0.0
3  1  0  0  1  0   -5.0       1     0.0
['BINARY', 4 rows, 10 samples, 5 variables]


## QA Max-Cut: Gset data

The Gset data consists of 71 weighted and unweighted graphs of different sizes (800 to 10,000 nodes) which can be dowloaded from [https://web.stanford.edu/~yyye/yyye/Gset](https://web.stanford.edu/~yyye/yyye/Gset).

Parse a Gset file into a NetworkX graph:

In [28]:
def parse_graph(filename):
    G = nx.Graph()
    
    with open(filename) as file:
        num_nodes, num_edges = map(int, file.readline().strip().split())
        
        for _ in range(num_edges):
            u, v, w = map(int, file.readline().strip().split())
            G.add_edge(u, v, weight=w)
        
    return num_nodes, num_edges, G

### G1 graph

Load the G1 graph:

In [43]:
num_nodes, num_edges, G = parse_graph('datasets/G1.txt')

assert num_nodes == G.number_of_nodes()
assert num_edges == G.number_of_edges()

print(f'G1 has {G.number_of_nodes()} nodes and {G.number_of_edges()} edges')

G1 has 800 nodes and 19176 edges


Construct the $\mathbf{Q}$ matrix (taking into account that G could be a weighted graph):

In [44]:
Q = defaultdict(int)

for i, j in G.edges:
    w = G.adj[i][j]['weight']
    
    Q[i, i] -= w
    Q[j, j] -= w
    Q[i, j] += 2 * w

In [55]:
chainstrength = 80
numruns = 10

In [56]:
%%time

sampler = EmbeddingComposite(DWaveSampler())
response = sampler.sample_qubo(Q,
                               chain_strength=chainstrength,
                               num_reads=numruns,
                               label='G1 - Maximum Cut')

ValueError: no embedding found

In [ ]:
dwave.inspector.show(response);